In [ ]:
from IPython import get_ipython
from IPython.display import display
import cv2
!pip install EasyOCR
!pip install fpdf
!pip install ultralytics
!pip install requests
import easyocr
import sqlite3
import time
import os
from datetime import datetime
from ultralytics import YOLO
from fpdf import FPDF
import requests
from google.colab.patches import cv2_imshow

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Load YOLO models
helmet_model = YOLO("/content/best (2).pt")  # Helmet detection model
license_plate_model = YOLO("/content/License_1.pt")  # License plate detection model

# Initialize EasyOCR for OCR
reader = easyocr.Reader(['en'])

# Database setup
DB_PATH = "violations.db"

def setup_database():
    """Creates the violations database if not exists"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS violations (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        license_plate TEXT,
                        timestamp TEXT,
                        image_path TEXT,
                        email_sent INTEGER DEFAULT 0
                      )''')
    conn.commit()
    conn.close()

setup_database()  # Ensure DB is ready

# Mailgun API Config
MAILGUN_API_KEY = "e4dc8f18c4a657e9423afbbfb19723dd-ac3d5f74-a2bbded9"
MAILGUN_DOMAIN = "sandboxbf8fcb9ef5ae4626abbc2eb685875452.mailgun.org"
RECIPIENT_EMAIL = "srijass2005@gmail.com"

def send_email(license_plate, snapshot_path):
    """Sends an e-challan email using Mailgun"""
    email_subject = f"E-Challan: Helmet Violation - {license_plate}"
    email_body = f"A helmet rule violation has been detected. License Plate: {license_plate}. Attached is the snapshot."

    files = [("attachment", open(snapshot_path, "rb"))]
    response = requests.post(
        f"https://api.mailgun.net/v3/{MAILGUN_DOMAIN}/messages",
        auth=("api", MAILGUN_API_KEY),
        files=files,
        data={"from": f"Traffic Authority <srijaknj9405@{MAILGUN_DOMAIN}>",
              "to": RECIPIENT_EMAIL,
              "subject": email_subject,
              "text": email_body})

    return response.status_code == 200

def generate_challan(license_plate, snapshot_path):
    """Generates an e-challan PDF for the violation"""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.cell(200, 10, txt="Traffic Violation E-Challan", ln=True, align="C")
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"License Plate: {license_plate}", ln=True, align="C")
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True, align="C")

    # Add snapshot
    pdf.image(snapshot_path, x=50, y=80, w=100)

    # Save PDF
    pdf_path = f"challans/{license_plate}_{int(time.time())}.pdf"
    os.makedirs("challans", exist_ok=True)
    pdf.output(pdf_path)
    return pdf_path

def process_frame(frame):
    """Processes a single frame to detect helmet violations and extract license plates"""
    results = helmet_model(frame)

    no_helmet_detected = False
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # Get class index
            print(f"🛑 Detected class: {cls}")  # Debugging output

            if cls == 0:  # Ensure '0' is 'No Helmet'
                no_helmet_detected = True

    if not no_helmet_detected:
        print("✅ No violation detected.")
        return None  # Skip further processing if no violation

    print("🚨 No helmet violation detected! Checking license plate...")

    # If violation detected, process license plate detection
    lp_results = license_plate_model(frame, conf=0.1)  # Lower threshold for better detection
    print(f"📸 License Plate Model Results: {lp_results}")  # Debugging output

    for lp_result in lp_results:
        for lp_box in lp_result.boxes:
            x1, y1, x2, y2 = map(int, lp_box.xyxy[0])  # Get bounding box
            plate_crop = frame[y1:y2, x1:x2]
            gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

            # Apply threshold for better OCR accuracy
            gray_plate = cv2.threshold(gray_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

            license_text = reader.readtext(gray_plate, detail=0)
            print(f"🔠 OCR Output: {license_text}")  # Debugging output

            if license_text:
                license_plate_str = license_text[0]  # Store license plate as a string
                return license_plate_str, plate_crop

    print("❌ No license plate detected.")
    return None  # No plate detected

def save_violation(license_plate, frame):
    """Saves violation data to DB and captures a snapshot"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    snapshot_path = f"snapshots/{license_plate}_{int(time.time())}.jpg"
    os.makedirs("snapshots", exist_ok=True)
    cv2.imwrite(snapshot_path, frame)

    print(f"📌 Saving {license_plate} to database...")
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO violations (license_plate, timestamp, image_path, email_sent) VALUES (?, ?, ?, ?)",
                   (license_plate, timestamp, snapshot_path, 0))
    conn.commit()
    conn.close()
    print(f"✅ Saved {license_plate} successfully!")

    return snapshot_path  # Return snapshot path for email function

def main():
    cap = cv2.VideoCapture("/content/WhatsApp Video 2025-02-22 at 12.09.58_da10c005.mp4")  # Change to 0 for webcam
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect helmet violation and extract license plate
        result = process_frame(frame)
        if result:
            license_plate, snapshot = result
            print(f"🚦 Violation detected: {license_plate}")

            # Save violation
            snapshot_path = save_violation(license_plate, frame)

            # Generate PDF
            pdf_path = generate_challan(license_plate, snapshot_path)

            # Send Email
            print(f"📧 Sending email for {license_plate}...")
            if send_email(license_plate, snapshot_path):
                print(f"✔ Email sent for {license_plate}")

                # Mark email as sent in database
                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()
                cursor.execute("UPDATE violations SET email_sent = 1 WHERE license_plate = ?", (license_plate,))
                conn.commit()
                conn.close()
            else:
                print(f"❌ Email sending failed!")

        # Show video feed
        cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow
        for i in range(100):  # Process 100 frames
          if cv2.waitKey(1) & 0xFF == ord('q'):
            break
          time.sleep(0.04)
        else:
            break
    cap.release()

if __name__ == "__main__":
    main()

def gradio_interface(video):
    return process_video(video)

demo = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Video(label="Upload Traffic Video"),
    outputs=gr.Textbox(label="Violation Report"),
    title="Helmet Rule Violation Detection",
    description="Upload a traffic video to detect helmet violations and process license plates for e-challans."
)

demo.launch()
